### 稳定性（Courant准则）
Courant准则定义如下：

$$
\epsilon = c \frac{ dt}{dx} \leq 1
$$

有了这个信息，我们可以计算最大可能和稳定的时间步长。

### 分析解

在下面的代码中，我们呈现了声波方程的分析解

$$
\partial_t^2 p(x,t) \ - \ c^2 \Delta p(x,t) \ = s(x,t)
$$

假设速度恒定为c，空间无限大。注意，在一维和二维情况下，这个方程在数学上等价于SH波传播问题（即，与源和接收器所在平面垂直极化的剪切波）。在三维情况下，它（仅）描述了压力（声）波。

非齐次偏微分方程（即，具有非零源项的方程）的解通常使用格林函数 $G(x, t; x_0, t_0)$ 的概念来开发。格林函数是特定偏微分方程的解，其中函数作为源项，其值在$(x, t)$处求解并在$(x_0, t_0)$处激活。因此，我们寻求解：

$$
\partial_t^2 G(x,t;x_0, t_0) \ - \ c^2 \Delta G(x,t;x_0, t_0) \ = \delta (x-x_0) \delta (t-t_0)
$$

其中 $\Delta$ 是拉普拉斯算子。我们回顾 $\delta-$函数的定义，它是广义函数，具有以下性质：

$$
\delta(x) = \left\{
\begin{array}{ll}
	\infty & x=0 \\
	0 & x\neq 0 	
\end{array}
\right.
$$

以及

$$
\int_{-\infty}^{\infty}\delta(x)dx\ = \ 1 \ , \   \int_{-\infty}^{\infty}f(x)\delta(x)dx\ = \ f(0) \ 
$$

在比较数值解和分析解时，接近 $\delta-$函数的极限的函数将变得非常重要。一个例子是矩形窗函数（boxcar function）：

$$
\delta_{bc}(x) = \left\{
\begin{array}{ll}
	1/dx & |x|\leq dx/2 \\
	0 & \text{其他} 	
\end{array}
\right.
$$

满足 $dx\rightarrow0$ 时的这些性质。这些函数用于适当缩放源项以获得正确的绝对幅度。

为了描述声波方程的分析解，我们还使用单位阶跃函数，也称为Heaviside函数，定义如下：

$$
H(x) = \left\{
\begin{array}{ll}
    0 & x<0 \\
	1 & x \geq 0 	
\end{array}
\right.
$$

Heaviside函数是 $\delta-$函数的积分（反之亦然，$\delta-$函数定义为Heaviside函数的导数）。在二维情况下，格林函数为

$$
G = \frac{1}{2\pi c^2}\frac{H(t-\frac{|r|}{c})}{\sqrt{t^2-\frac{r^2}{c^2}}}
$$

$$
r = \sqrt{x^2+y^2}
$$

在二维中发生了特殊情况。冲击源导致具有随时间减小的尾波的波形。这是事实上源是线源的结果。从计算的角度来看，这非常重要。在二维笛卡尔坐标中，数值解不能直接与通常具有点源的观测进行比较。

### 数值解（有限差分法）

在二维中，常密度声波方程为

$$
\partial^2_t{p}(x,z,t) \ = \ c^2 (\partial_x^2 p(x,z,t) + \partial_z^2 p(x,z,t)) \ + s(x,z,t)
$$

其中选择 $z$-坐标，因为在许多应用中将考虑 $x-z$ 平面为垂直平面，$z$ 为深度坐标。根据以上发展，我们用以下形式离散化时空：

$$
p(x,z,t) \ \rightarrow \ p^n_{i,j} \ = \ p(n dt, i dx, j dz) \ .
$$

使用时间二阶导数的三点算子，我们得到外推方案

$$
 \frac{p_{i,j}^{n+1} - 2 p_{i,j}^n + p_{i,j}^{n-1}}{dt^2} \ = \ c^2 ( \partial_x^2 p + \partial_z^2 p) \ + s_{i,j}^n
$$

在右边，假定空间和时间依赖性，并且偏导数近似为

\begin{equation}
\begin{split}
\partial_x^2 p \ &= \ \frac{p_{i+1,j}^{n} - 2 p_{i,j}^n + p_{i-1,j}^{n}}{dx^2} \\ 
\partial_z^2 p \ &= \ \frac{p_{i,j+1}^{n} - 2 p_{i,j}^n + p_{i,j-1}^{n}}{dz^2}  \ . 
\end{split}
\end{equation}

注意对于规则的二维网格，$dz=dx$。

### 分析和数值对比
下面的代码使用3点差分算子。将数值模拟的结果与分析解进行比较。

### 高阶算子
通过添加使用5点差分算子的选项来扩展代码。5点权重为：

$$
[-1/12, 4/3, -5/2, 4/3, -1/12]/dx^2
$$. 

In [1]:
# Import Libraries (PLEASE RUN THIS CODE FIRST!) 
# ----------------------------------------------
import numpy as np
import matplotlib
# Show Plot in The Notebook
matplotlib.use("nbagg")
import matplotlib.pyplot as plt

# Sub-plot Configuration
# ----------------------
from matplotlib import gridspec 
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Ignore Warning Messages
# -----------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Parameter Configuration 
# -----------------------

nx   = 500          # number of grid points in x-direction
nz   = nx           # number of grid points in z-direction
# Note: regular 2D grid, dz = dx
dx   = 1.           # grid point distance in x-direction
dz   = dx           # grid point distance in z-direction
c0   = 580.         # wave velocity in medium (m/s)
isx  = 200          # source location in grid in x-direction
isz  = isx          # source location in grid in z-direction
irx  = 300          # receiver location in grid in x-direction
irz  = irx          # receiver location in grid in z-direction
nt   = 502          # maximum number of time steps
dt   = 0.0050       # time step
f0   = 25. # dominant frequency of the source (Hz)
t0   = 2. / f0 # source time shift
op   = 3            # length of finite-difference operator (3 or 5)

print('Source frequency =', f0, 'Hz')

# CFL Stability Criterion
# -----------------------
eps  = c0 * dt / dx # epsilon value
print('Stability criterion =', eps)

# Initialize Empty Pressure
# -------------------------
p    = np.zeros((nz, nx)) # p at time n (now)
pold = np.zeros((nz, nx)) # p at time n-1 (past)
pnew = np.zeros((nz, nx)) # p at time n+1 (present)
d2px = np.zeros((nz, nx)) # 2nd space derivative of p in x-direction
d2pz = np.zeros((nz, nx)) # 2nd space derivative of p in z-direction

# Initialize Velocity Model (assume homogeneous model)
# ----------------------------------------------------
c    = np.zeros((nz, nx))
c    = c + c0             # initialize wave velocity in model

# Initialize Grid
x    = np.arange(nx)
x    = x * dx             # coordinate in x-direction
z    = np.arange(nz)
z    = z * dz             # coordinate in z-direction

# Initialize Empty Seismogram
# ---------------------------
seis = np.zeros(nt)

# Fontsize
# fs=20

Source frequency = 25.0 Hz
Stability criterion = 2.9


In [3]:
# Initialize Source Time Function 
# -------------------------
# Source time function (Gaussian)
# -------------------------------
src  = np.zeros(nt + 1)
time = np.linspace(0 * dt, nt * dt, nt)
# 1st derivative of a Gaussian
src  = -8. * (time - t0) * f0 * (np.exp(-1.0 * (4*f0) ** 2 * (time - t0) ** 2))

In [4]:
# Plot Source Time Function 
# -------------------------

# Plot Position Configuration
# ---------------------------
plt.ion()
fig1 = plt.figure(figsize=(10, 6))
gs1  = gridspec.GridSpec(1, 2, width_ratios=[1, 1], hspace=0.3, wspace=0.3)

# Plot Source Time Function
# -------------------------
ax1  = plt.subplot(gs1[0])
ax1.plot(time, src) # plot source time function
ax1.set_title('Source Time Function')
ax1.set_xlim(time[0], time[-1])
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Amplitude')

# Plot Source Spectrum
# --------------------
ax2  = plt.subplot(gs1[1])
spec = np.fft.fft(src) # source time function in frequency domain
freq = np.fft.fftfreq(spec.size, d = dt) # time domain to frequency domain
ax2.plot(np.abs(freq), np.abs(spec)) # plot frequency and amplitude
ax2.set_xlim(0, 250) # only display frequency from 0 to 250 Hz
ax2.set_title('Source Spectrum')
ax2.set_xlabel('Frequency (Hz)')

ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")

plt.show()

<IPython.core.display.Javascript object>

In [5]:
# Plot Snapshot & Seismogram (PLEASE RERUN THIS CODE AGAIN AFTER SIMULATION!) 
# ---------------------------------------------------------------------------

# Analytical Solution 
# -------------------
G    = time * 0.
r    = np.sqrt((x[isx] - x[irx]) ** 2 + (z[isz] - z[irz]) ** 2)

for it in range(nt): # Calculate Green's function
    #if ((time[it] - np.abs(x[irx] - x[isx]) / c0) >= 0):
    if ((time[it] - r/c0) >= 0):
        G[it] = (1. / (2 * np.pi * c0 ** 2)) \
        * (1. / np.sqrt((time[it] ** 2) - (r ** 2 / (c0 ** 2))))
Gc   = np.convolve(G, src * dt)
Gc   = Gc[0:nt]
lim  = Gc.max() # get limit value from maximum amplitude of analytical solution

# Plot Position Configuration
# ---------------------------
plt.ion()
fig2 = plt.figure(figsize=(10, 6))
gs2  = gridspec.GridSpec(1,2,width_ratios=[1,1],hspace=0.3, wspace=0.3)

# Plot 2D Wave Propagation
# ------------------------
# Note: comma is needed to update the variable
ax3  = plt.subplot(gs2[0])
leg1,= ax3.plot(isx, isz, 'r*', markersize=11) # plot position of the source in model
leg2,= ax3.plot(irx, irz, 'k^', markersize=8)  # plot position of the receiver in model
im3  = ax3.imshow(p, vmin=-lim, vmax=+lim, interpolation="nearest", cmap=plt.cm.RdBu)
div  = make_axes_locatable(ax3)
cax  = div.append_axes("right", size="5%", pad=0.05) # size & position of colorbar
fig2.colorbar(im3, cax=cax) # plot colorbar
ax3.set_title('Time Step (nt) = 0')
ax3.set_xlim(0, nx)
ax3.set_ylim(0, nz)
ax3.set_xlabel('nx')
ax3.set_ylabel('nz')
ax3.legend((leg1, leg2), ('Source', 'Receiver'), loc='upper right', fontsize=10, numpoints=1)

# Plot Seismogram 
# ---------------
# Note: comma is needed to update the variable
ax4  = plt.subplot(gs2[1])
up41,= ax4.plot(time, seis) # update seismogram each time step
up42,= ax4.plot([0], [0], 'r|', markersize=15) # update time step position
ax4.set_xlim(time[0], time[-1])
ax4.set_title('Seismogram')
ax4.set_xlabel('Time [s]')
ax4.set_ylabel('Amplitude')
leg3,= ax4.plot(0,0,'r--',markersize=1)
leg4,= ax4.plot(0,0,'b-',markersize=1)
ax4.legend((leg3, leg4), ('Analytical', 'FD'), loc='upper right', fontsize=10, numpoints=1)

ax4.yaxis.tick_right()
ax4.yaxis.set_label_position("right")

plt.plot(time,Gc,'r--')
plt.show()

<IPython.core.display.Javascript object>

In [6]:
# 2D Wave Propagation (Finite Difference Solution) 
# ------------------------------------------------


# Calculate Partial Derivatives
# -----------------------------
for it in range(nt):
    if op == 3: # use 3 point operator FD scheme
        for i in range(1, nx - 1):
                d2px[i, :] = (p[i - 1, :] - 2 * p[i, :] + p[i + 1, :]) / dx ** 2 
        for j in range(1, nz - 1):
                d2pz[:, j] = (p[:, j - 1] - 2 * p[:, j] + p[:, j + 1]) / dz ** 2 
    
    if op == 5: # use 5 point operator FD scheme
        #-----------------------------------------------#
        #     IMPLEMENT 5 POINT OPERATOR CODE HERE!     #
        #-----------------------------------------------#
        ### created by llp 2023_10_9_14:42
        for i in range(1, nx - 1):
            d2px[i, :] = (-1/12 * p[i - 2, :] + 4/3 * p[i - 1, :] - 5/2 * p[i, :] + 4/3 * p[i + 1, :] - 1/12 * p[i + 2, :]) / dx ** 2
        for j in range(1, nz - 1):
            d2px[:, j] = (-1/12 * p[:, j - 2] + 4/3 * p[:, j - 1] - 5/2 * p[:, j] + 4/3 * p[:, j + 1] - 1/12 * p[:, j + 2]) / dx ** 2
    
    # Time Extrapolation
    # ------------------
    pnew = 2 * p - pold + (c ** 2) * (dt ** 2) * (d2pz + d2px)
    
    # Add Source Term at isz and isx
    # ------------------------------
    # Absolute pressure w.r.t analytical solution
    pnew[isz, isx] = pnew[isz, isx] + src[it] / (dx * dz) * (dt ** 2) 
    
    # Remap Time Levels
    # -----------------
    pold, p = p, pnew
    
    # Output Seismogram
    # -----------------
    seis[it] = p[irz, irx]
    
    # Update Data for Wave Propagation Plot
    # -------------------------------------
    idisp = 5 # display frequency
    if (it % idisp) == 0:
        ax3.set_title('Time Step (nt) = %d' % it)
        ax3.imshow(p,vmin=-lim, vmax=+lim, interpolation="nearest", cmap=plt.cm.RdBu)
        up41.set_ydata(seis)
        up42.set_data(time[it], seis[it])
        # Uncoment for bigger fontsize - also uncoment the Fontsize in the cell parameters
        #for item in ([ax3.title, ax3.xaxis.label, ax3.yaxis.label] +
        #     ax3.get_xticklabels() + ax3.get_yticklabels()):
        #     item.set_fontsize(fs)
        #for item in ([ax4.title, ax4.xaxis.label, ax4.yaxis.label] +
        #     ax4.get_xticklabels() + ax4.get_yticklabels()):
        #     item.set_fontsize(fs)
        plt.gcf().canvas.draw()

<IPython.core.display.Javascript object>